In [ ]:
import logging
import pickle
from multiprocessing import Pool
from pathlib import Path

import pandas as pd

from paddel.preprocessing.poses import extract_poses_df

In [ ]:
logging.basicConfig(level=logging.INFO)

min_detection_seconds = 15

data_dir = Path("../data")
file_features_path = data_dir / "file_features_df.pkl"
y_path = data_dir / "y.pkl"
poses_path = data_dir / "poses_df.pkl"

Load file features

In [ ]:
with open(file_features_path, "rb") as f:
    file_features_df = pickle.load(f)

with open(y_path, "rb") as f:
    y = pickle.load(f)

Extract poses

In [ ]:
with Pool() as p:
    poses_dfs = p.map(extract_poses_df, file_features_df.to_dict(orient="index").items())

Concatenate dataframes and filter when not enough poses detected

In [ ]:
all_poses_df = pd.DataFrame()

for poses_df in poses_dfs:
    index = poses_df.loc[0]["id"]
    framerate = file_features_df.loc[index]["framerate"]
    frame_count = len(poses_df.index)

    if frame_count < framerate * min_detection_seconds:
        file_features_df.drop(index, inplace=True)
        y.drop(index, inplace=True)
        continue

    all_poses_df = pd.concat([all_poses_df, poses_df], axis=0, ignore_index=True, copy=False)

Save poses and update file features and target

In [ ]:
with open(poses_path, "wb") as f:
    pickle.dump(all_poses_df, f)

with open(file_features_path, "wb") as f:
    pickle.dump(file_features_df, f)

with open(y_path, "wb") as f:
    pickle.dump(y, f)